
## Steps

- 1st : `DONE` Scrap song name, author, hindi_text, translation text
- 2st : `DONE` convert hindi to iast 
- 3nd : `DONE` Create new song in db using above values
- 4rd : `DONE` get dictionary for greenmesg
- 5th : `DONE` create dic entry in dictionary.db
- 6th : `TODO` get line by line iast song, dict value
- 7th : `TODO` get line by line : synonyms
- 8th : `TODO` update song synonyms db

In [1]:
import os
from slokabase.SqliteModel import get_create_table_query, get_insert_query, get_read_query, get_update_query, get_delete_query
from slokabase.SqliteModel import SqliteModel
db_name = 'dictionary.db'
db_path = os.path.join(os.getcwd(),db_name)
DictonaryWord_sql = SqliteModel(db_path,'DictWord')
DictonaryMeaning_sql = SqliteModel(db_path,'DictMeaning')

db_name = 'slokabase.db'
db_path = os.path.join(os.getcwd(),db_name)
SongIndex_sql = SqliteModel(db_path,'SongIndex')
mySongs_sql = SqliteModel(db_path,'Songs')

In [175]:
# url = 'https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php'
# url = 'https://greenmesg.org/stotras/annapoorna/annapoorna_stotram.php'
# url = 'https://greenmesg.org/stotras/vishnu/venkatesa_suprabhatam.php'
# url='https://greenmesg.org/stotras/lakshmi/kanakadhara_stotram.php'
# url='https://greenmesg.org/stotras/rama/nama_ramayana.php'
url = "https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php"
# "https://greenmesg.org/stotras/saraswati/sharada_bhujangam.php"
# "https://greenmesg.org/stotras/kali/kalika_ashtakam.php"
# "https://greenmesg.org/stotras/durga/bhavani_ashtakam.php"
# "https://greenmesg.org/stotras/lalita/lalita_panchakam.php"
# "https://greenmesg.org/stotras/narasimha/lakshmi_nrisimha_karavalambam.php"
# "https://greenmesg.org/stotras/ganesha/ganesha_pancharatnam.php"
# 'https://greenmesg.org/stotras/brahman/nirvana_shatkam.php'
# 'https://greenmesg.org/stotras/kamakshi/kamakshi_stotram.php'
# url ='https://greenmesg.org/stotras/gayatri/gayatri_mantra.php'
# url ='javascript:showMng()'
# 'https://greenmesg.org/stotras/gayatri/gayatri_mantra.php'

In [176]:
from slokabase.hindi2english import hindi2iast 
# hindi2iast(dict_entry[0]['hindi']).strip()
from bs4 import BeautifulSoup
import requests

In [180]:
source = requests.get(url).text
soup = BeautifulSoup(source)
song_name, song_author = get_name_author(soup)
hindi_text       = get_sanskrit_text(soup)
# hindi_text = hindi_text.replace("_",'')
iast_text = hindi2iast(hindi_text)
translation_text = get_translation(soup)

# print(song_name, song_author)
# print(hindi_text)
# print(iast_text.strip())
# print(translation_text.strip())

In [181]:
print(song_name, song_author)

# print(translation_text)
# print(iast_text)
# print(hindi_text)
print(len(hindi_text.split('\n\n')), len( translation_text.split('\n\n') )  )

Shiva Panchakshara Stotram Sri Adi Shankaracharya
6 6


In [183]:
# SongIndex_sql.create_entry(**song_info)
song_info = {}
song_info['song_name'] = song_name # 'My Dasavatara Stotram'
song_info['song_short_name'] = song_name.replace(" ","") # 'MyDasaVatStot'
song_info['sloka_statues'] = 1
song_info['division_no'] = 0
song_info['total_slokas'] = len(hindi_text.split('\n\n'))
# song_info['groups']
song_info['devotion_god'] = 'Devi Kamakshi'
song_info['author']= song_author 
# song_info['describtion']=  
song_info['other_links '] = url

if len(SongIndex_sql.read_entry(**song_info)) ==0:
    SongIndex_sql.create_entry(**song_info)    
    print('Create Entry')        

song_idx = SongIndex_sql.read_entry( *['song_idx'], **song_info)[0]['song_idx'] # >>> 5
song_info['song_idx']= song_idx 
print('song_info: ',song_info)

Create Entry
song_info:  {'song_name': 'Shiva Panchakshara Stotram', 'song_short_name': 'ShivaPanchaksharaStotram', 'sloka_statues': 1, 'division_no': 0, 'total_slokas': 6, 'devotion_god': 'Devi Kamakshi', 'author': 'Sri Adi Shankaracharya', 'other_links ': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'song_idx': 90}


In [184]:
song = {}
print(type(song))
song['sloka_eng'] = iast_text
song['sloka_hindi'] = hindi_text
song['translation'] = translation_text
SongIndex_sql.read_entry(*['song_idx'], **song_info)[0]['song_idx']

<class 'dict'>


90

In [185]:
mysong = {} #dict()
mySongs_sql = SqliteModel(db_path,'Songs')
mysong ['song_idx'] = SongIndex_sql.read_entry(*['song_idx'], **song_info)[0]['song_idx']

print(song_name)
list_sloka = song['sloka_eng'].split('\n\n')
print("Sloka English")
for idx, sloka in enumerate(list_sloka):
    print(idx)
    mysong['slokas_no']   = idx+1
    mysong['sloka_eng']   = sloka.strip()
    mySongs_sql.create_entry(**mysong)
del mysong['sloka_eng']

list_sloka = song['sloka_hindi'].split('\n\n')
print("Hindi")
for idx, sloka in enumerate(list_sloka):
    print(idx)
    mysong['slokas_no']   = idx+1
    mysong['sloka_hindi']   = sloka.strip()
    mySongs_sql.update_entry( *['song_idx','slokas_no']  , **mysong)
del mysong['sloka_hindi']

list_sloka = song['translation'].split('\n\n')    
print("Translation")
for idx, sloka in enumerate(list_sloka):
    print(idx)
    mysong['slokas_no']   = idx+1
    mysong['translation']   = sloka.strip()
    mySongs_sql.update_entry( *['song_idx','slokas_no']  , **mysong)
del mysong['translation']

Shiva Panchakshara Stotram
Sloka English
0
1
2
3
4
5
Hindi
0
1
2
3
4
5
Translation
0
1
2
3
4
5


In [186]:
def get_name_author(soup):
    container = soup.find("div", {"id": "Container"})
    song_name = container.h1.text.split('-')[0].strip() # 'Devi Aparadha Kshamapana Stotram'
    if '- composed by' in container.text: 
        song_author = container.text.split('- composed by')[1].split('\n')[0].strip() # 'Sri Adi Shankaracharya'
    else:
        song_author=None
    return song_name, song_author 

def get_sanskrit_text(soup):
    hindi_text = ''
    for verse in soup.select('span.Sanskrit'):
        for br in verse.find_all("br"):
            br.replace_with("\n")               
        hindi_text += verse.text
        hindi_text += '\n'
    
    hindi_text =hindi_text.strip()
    return hindi_text
    
def get_translation(soup):
    verse = soup.select('span.Sanskrit')[0]
    sub_containt = verse.parent#.text.split('||')
    for br in sub_containt.find_all("br"):
        br.replace_with("\n")
    output = ''
    for para in sub_containt.text.split('\n\n'):
        if 'Meaning' in para:
            # print('Meaning in para',para)
            lines = para.replace("Meaning:\n","").split('\n')
            for line in lines:
                split_list = line.split(':')
                prefix = split_list[0].strip()
                del split_list[0]
                text = '\n'.join(split_list)
                prefix_no = prefix.split('.')
                if len(prefix_no) ==2 and prefix_no[0].isnumeric() and prefix_no[1].isnumeric():
                    # print(text)
                    output +=text.strip() + '\n'
                else: 
                    # print(prefix,text) 
                    output += prefix + " " + text.strip() + '\n'
            # print()
            output +='\n'
    return output.strip()    

def scrap_word(main_word,sub_word_list):
    dict_entry= {} # dict()
    main_word = BeautifulSoup(main_word).text
    words_list = main_word.split("__")
    # print(words_list,main_word,sub_word_list)
    dict_entry['hindi'] = words_list[1].split('(')[0].strip()
    # p_iast_word = words_list[1].split('(')[1].split(')')[0].strip()
    dict_entry['sandhi']= words_list[1].split('(')[1].split(')')[0].strip()
    dict_entry['meaning'] = words_list[1].split('(')[1].split(')')[1].replace(':','').strip()
    # print(f'''{sanskrit_word}({p_iast_word}) = {meaning_word}''')
    if isinstance(sub_word_list, list):
        output_list = []
        for sub_word in sub_word_list:
            temp_dict = {} # dict()            
            temp_dict['hindi'] = BeautifulSoup( sub_word).text.split('(')[0]
            temp_dict['sandhi']=BeautifulSoup( sub_word).text.split('(')[1].split(')')[0].strip()
            temp_dict['meaning']=BeautifulSoup( sub_word).text.split('(')[1].split(')')[1].replace('=','').strip()
            # print()
            # print(f'''>>>{sanskrit_word}({p_iast_word}) = {meaning_word}''')
            output_list.append(temp_dict)
        dict_entry['subwords'] = output_list
    return dict_entry

def get_query_dic(temp_dict,url):
    dicword={}
    dicword['word'] =hindi2iast(temp_dict['hindi']).strip() # word
    dicword['hindi'] =temp_dict['hindi']                     # hindi 
    dicword['sandhi'] =temp_dict['sandhi']                    # sandhi
    if 'subwords' in temp_dict.keys() : 
        dicword['subwords'] = ''
        for dict in temp_dict['subwords']:
            dicword['subwords']+=hindi2iast(dict['hindi']).strip()+'-'
        dicword['subwords'] = dicword['subwords'][:-1]
    # print(dicword) 
    # # form sl
    dicmeaning = {}
    dicmeaning['word'] = dicword['word']
    dicmeaning['source'] = url
    dicmeaning['meaning_value'] = temp_dict['meaning']
    # print(dicmeaning)
    return dicword, dicmeaning
def create_dict_entry(DictonaryMeaning_sql, DictonaryWord_sql, dicword, dicmeaning):
    if len(DictonaryWord_sql.read_entry(word=dicword['word'])) ==1:
        if len(DictonaryWord_sql.read_entry(**dicword)) ==0:
            # print(f'word:{dicword["hindi"]} already exit but what about hindi, sandi,subword')
            DictonaryWord_sql.update_entry(*['word'],**dicword)
            # (**dicword)       
    else:
        DictonaryWord_sql.create_entry(**dicword)
    if len(DictonaryMeaning_sql.read_entry(word=dicmeaning['word'],meaning_value=dicmeaning['meaning_value'])) ==1:
        # already existing
        pass
    else : 
        last_idx = len(DictonaryMeaning_sql.read_entry(word=dicmeaning['word']))
        dicmeaning['meaning_idx'] = last_idx +1
        DictonaryMeaning_sql.create_entry(**dicmeaning)

def get_dic_list(line_list):
    dic_list=[]
    for idx, item in enumerate(line_list):
        if item == '' or item is None:
            continue
        if '<br/>' not in item:
            main_word = item
            sub_word =''
            dict_entry=scrap_word(main_word,sub_word)
            dic_list.append(dict_entry)
        else:
            all_words =item.split('<br/>')
            main_word = all_words[0]
            del all_words[0]
            sub_word_list = all_words
            print(main_word,sub_word_list)
            dict_entry=scrap_word(main_word,sub_word_list)
            dic_list.append(dict_entry)
    return dic_list

# dic_list = get_dic_list(line_list)    

In [187]:
# # print
# print(get_name_author(soup))
# print(get_sanskrit_text(soup))
# print(get_translation(soup))

# curl 'https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0' -H 'Accept: */*' -H 'Accept-Language: en-US,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br' -H 'DNT: 1' -H 'Connection: keep-alive' -H 'Referer: https://greenmesg.org/stotras/durga/devi_aparadha_kshamapana_stotram.php' -H 'Sec-Fetch-Dest: empty' -H 'Sec-Fetch-Mode: cors' -H 'Sec-Fetch-Site: same-origin'

In [188]:
# print(url)
# print(url.replace('stotras','dictionary/stotras').replace('php','txt'))
# print('https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt')

In [189]:
# x = requests.get('https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt')
# x =requests.request('GET','https://greenmesg.org/dictionary/stotras/durga/devi_aparadha_kshamapana_stotram.txt')


In [190]:
# for item in line_list:
#     print(BeautifulSoup(item))


In [191]:
x =requests.request('GET', url.replace('stotras','dictionary/stotras').replace('php','txt') )
line_list = x.text.split('\r\n')
line_list

["naagen-d-rahaaraaya__<span class='mword'>&#x0928;&#x093E;&#x0917;&#x0947;&#x0928;&#x094D;&#x0926;&#x094D;&#x0930;&#x0939;&#x093E;&#x0930;&#x093E;&#x092F;</span> (<span class='trans1'>Naagendra-Haaraaya</span>): <span class='desc'>(Salutations to Lord Shiva)</span> Who has the most <span class='kwi'>excellent</span> of <span class='kwi'>Snakes</span> as His <span class='kwi'>Garland</span> <br/> <span class='mng'>&#x0928;&#x093E;&#x0917;</span> (<span class='trans2'>Naaga</span>) = Snake, Serpent <br/> <span class='mng'>&#x0907;&#x0928;&#x094D;&#x0926;&#x094D;&#x0930;</span> (<span class='trans2'>Indra</span>) = Best, Excellent, Chief <br/> <span class='mng'>&#x0939;&#x093E;&#x0930;</span> (<span class='trans2'>Haara</span>) = Garland",
 "t-rilocanaaya__<span class='mword'>&#x0924;&#x094D;&#x0930;&#x093F;&#x0932;&#x094B;&#x091A;&#x0928;&#x093E;&#x092F;</span> (<span class='trans1'>Tri-Locanaaya</span>): <span class='desc'>(Salutations to Lord Shiva)</span> Who has <span class='kwi'>Th

In [192]:

# for dict_entry in get_dic_list(line_list):
dict_entry = get_dic_list(line_list)[4]
print('dict_entry: ',dict_entry)
dicword={}
dicword['word'] =hindi2iast(dict_entry['hindi']).strip() # word
dicword['hindi'] =dict_entry['hindi']                     # hindi 
dicword['sandhi'] =dict_entry['sandhi']                    # sandhi
if 'subwords' in dict_entry.keys() : 
    dicword['subwords'] = ''
    for dict in dict_entry['subwords']:
        dicword['subwords']+=hindi2iast(dict['hindi']).strip()+'-'
    dicword['subwords'] = dicword['subwords'][:-1]
print('dicword: ',dicword) 
dicword, dicmeaning = get_query_dic(dict_entry,url)
print('get_query_dic=> dicword',dicword)
print('get_query_dic=> dicmeaning',dicmeaning)

naagen-d-rahaaraaya__<span class='mword'>&#x0928;&#x093E;&#x0917;&#x0947;&#x0928;&#x094D;&#x0926;&#x094D;&#x0930;&#x0939;&#x093E;&#x0930;&#x093E;&#x092F;</span> (<span class='trans1'>Naagendra-Haaraaya</span>): <span class='desc'>(Salutations to Lord Shiva)</span> Who has the most <span class='kwi'>excellent</span> of <span class='kwi'>Snakes</span> as His <span class='kwi'>Garland</span>  [" <span class='mng'>&#x0928;&#x093E;&#x0917;</span> (<span class='trans2'>Naaga</span>) = Snake, Serpent ", " <span class='mng'>&#x0907;&#x0928;&#x094D;&#x0926;&#x094D;&#x0930;</span> (<span class='trans2'>Indra</span>) = Best, Excellent, Chief ", " <span class='mng'>&#x0939;&#x093E;&#x0930;</span> (<span class='trans2'>Haara</span>) = Garland"]
t-rilocanaaya__<span class='mword'>&#x0924;&#x094D;&#x0930;&#x093F;&#x0932;&#x094B;&#x091A;&#x0928;&#x093E;&#x092F;</span> (<span class='trans1'>Tri-Locanaaya</span>): <span class='desc'>(Salutations to Lord Shiva)</span> Who has <span class='kwi'>Three Eyes

In [193]:
# dict_entry= [{'hindi': 'पापघ्नी', 'sandhi': 'Paapa-Ghnii', 'meaning': 'Remover of Sins', 
#  'subwords': [{'hindi': ' पाप ', 'sandhi': 'Paapa', 'meaning': 'Sin'}, 
#              {'hindi': ' घ्न ', 'sandhi': 'Ghna', 'meaning': 'Remover, Killer'}]},]
# # {'hindi': 'महादेवि', 'sandhi': 'Mahaadevi', 'meaning': 'Mahadevi, the great Goddess'}]
# temp_dict = dict_entry[0]



In [194]:
# dic_song_list = []
# for item in dic_list:
#     if 'subwords'  in item.keys() :
#         temp_list = []
#         for sub_item in item['subwords']:
#             temp_list.append((hindi2iast(sub_item['hindi']).strip(),sub_item['meaning'])) 
#         dic_song_list.append(temp_list)

#     dic_song_list.append((hindi2iast(item['hindi']).strip(),item['meaning']))        
# print(dic_song_list)

#### 1st Step: WebScrapping the dictionary form greenmesg

In [195]:
for dict_entry in get_dic_list(line_list):
    dicword, dicmeaning = get_query_dic(dict_entry,url)    
    create_dict_entry(DictonaryMeaning_sql, DictonaryWord_sql, dicword, dicmeaning)
    print('#')
    print('# mainword:',dicword, dicmeaning ) 
    if 'subwords' in dict_entry.keys() :     
        for dict_subword in dict_entry['subwords']:
            dicword, dicmeaning = get_query_dic(dict_subword,url)    
            create_dict_entry(DictonaryMeaning_sql, DictonaryWord_sql, dicword, dicmeaning)
            print('\t# subword:',dicword, dicmeaning)
            print(dicword, dicmeaning ) 

naagen-d-rahaaraaya__<span class='mword'>&#x0928;&#x093E;&#x0917;&#x0947;&#x0928;&#x094D;&#x0926;&#x094D;&#x0930;&#x0939;&#x093E;&#x0930;&#x093E;&#x092F;</span> (<span class='trans1'>Naagendra-Haaraaya</span>): <span class='desc'>(Salutations to Lord Shiva)</span> Who has the most <span class='kwi'>excellent</span> of <span class='kwi'>Snakes</span> as His <span class='kwi'>Garland</span>  [" <span class='mng'>&#x0928;&#x093E;&#x0917;</span> (<span class='trans2'>Naaga</span>) = Snake, Serpent ", " <span class='mng'>&#x0907;&#x0928;&#x094D;&#x0926;&#x094D;&#x0930;</span> (<span class='trans2'>Indra</span>) = Best, Excellent, Chief ", " <span class='mng'>&#x0939;&#x093E;&#x0930;</span> (<span class='trans2'>Haara</span>) = Garland"]
t-rilocanaaya__<span class='mword'>&#x0924;&#x094D;&#x0930;&#x093F;&#x0932;&#x094B;&#x091A;&#x0928;&#x093E;&#x092F;</span> (<span class='trans1'>Tri-Locanaaya</span>): <span class='desc'>(Salutations to Lord Shiva)</span> Who has <span class='kwi'>Three Eyes

#
# mainword: {'word': 'nāgēndrahārāya', 'hindi': 'नागेन्द्रहाराय', 'sandhi': 'Naagendra-Haaraaya', 'subwords': 'nāga-indra-hāra'} {'word': 'nāgēndrahārāya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': '', 'meaning_idx': 1}
	# subword: {'word': 'nāga', 'hindi': ' नाग ', 'sandhi': 'Naaga'} {'word': 'nāga', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Snake, Serpent'}
{'word': 'nāga', 'hindi': ' नाग ', 'sandhi': 'Naaga'} {'word': 'nāga', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Snake, Serpent'}
	# subword: {'word': 'indra', 'hindi': ' इन्द्र ', 'sandhi': 'Indra'} {'word': 'indra', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Best, Excellent, Chief', 'meaning_idx': 7}
{'word': 'indra', 'hindi': ' इन्द्र ', 'sandhi': 'Indra'} {'word': 'indra', 'source': 'https://greenmesg.org/s

#
# mainword: {'word': 'śivāya', 'hindi': 'शिवाय', 'sandhi': 'Shivaaya', 'subwords': 'śiva'} {'word': 'śivāya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': '', 'meaning_idx': 1}
	# subword: {'word': 'śiva', 'hindi': ' शिव ', 'sandhi': 'Shiva'} {'word': 'śiva', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Lord Shiva, Auspicious', 'meaning_idx': 7}
{'word': 'śiva', 'hindi': ' शिव ', 'sandhi': 'Shiva'} {'word': 'śiva', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Lord Shiva, Auspicious', 'meaning_idx': 7}
#
# mainword: {'word': 'mandākinīsalilacandanacarcitāya', 'hindi': 'मन्दाकिनीसलिलचन्दनचर्चिताय', 'sandhi': 'Mandaakinii-Salila-Candana-Carcitaaya', 'subwords': 'mandākinī-salila-candana-carcita'} {'word': 'mandākinīsalilacandanacarcitāya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_valu

#
# mainword: {'word': 'gaurīvadanābjavr̥nda', 'hindi': 'गौरीवदनाब्जवृन्द', 'sandhi': 'Gaurii-Vadana-Abja-Vrnda', 'subwords': 'gaurī-vadana-abja-vr̥nda'} {'word': 'gaurīvadanābjavr̥nda', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': '', 'meaning_idx': 1}
	# subword: {'word': 'gaurī', 'hindi': ' गौरी ', 'sandhi': 'Gaurii'} {'word': 'gaurī', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Devi Gauri [Parvati]', 'meaning_idx': 2}
{'word': 'gaurī', 'hindi': ' गौरी ', 'sandhi': 'Gaurii'} {'word': 'gaurī', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Devi Gauri [Parvati]', 'meaning_idx': 2}
	# subword: {'word': 'vadana', 'hindi': ' वदन ', 'sandhi': 'Vadana'} {'word': 'vadana', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Face'}
{'word': 'vadana', 'hindi': ' वदन ', 'sandhi': 'Vadana'} {'wo

	# subword: {'word': 'ārya', 'hindi': ' आर्य ', 'sandhi': 'Aarya'} {'word': 'ārya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Respectable, Honourable', 'meaning_idx': 1}
{'word': 'ārya', 'hindi': ' आर्य ', 'sandhi': 'Aarya'} {'word': 'ārya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Respectable, Honourable', 'meaning_idx': 1}
#
# mainword: {'word': 'mūnīndradēvārcitaśēkharāya', 'hindi': 'मूनीन्द्रदेवार्चितशेखराय', 'sandhi': 'Muuni-Indra-Deva-Aarcita-Shekharaaya', 'subwords': 'muni-indra-dēva-arcita-śēkhara'} {'word': 'mūnīndradēvārcitaśēkharāya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': '', 'meaning_idx': 1}
	# subword: {'word': 'muni', 'hindi': ' मुनि ', 'sandhi': 'Muni'} {'word': 'muni', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Sage', 'meaning_idx': 3}
{'word': '

#
# mainword: {'word': 'pinākahastāya', 'hindi': 'पिनाकहस्ताय', 'sandhi': 'Pinaaka-Hastaaya', 'subwords': 'pināka-hasta'} {'word': 'pinākahastāya', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': '', 'meaning_idx': 1}
	# subword: {'word': 'pināka', 'hindi': ' पिनाक ', 'sandhi': 'Pinaaka'} {'word': 'pināka', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Bow of Lord Shiva, Trident of Lord Shiva', 'meaning_idx': 1}
{'word': 'pināka', 'hindi': ' पिनाक ', 'sandhi': 'Pinaaka'} {'word': 'pināka', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Bow of Lord Shiva, Trident of Lord Shiva', 'meaning_idx': 1}
	# subword: {'word': 'hasta', 'hindi': ' हस्त ', 'sandhi': 'Hasta'} {'word': 'hasta', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Hand'}
{'word': 'hasta', 'hindi': ' हस्त ', 'sandhi': 'Hasta'

#
# mainword: {'word': 'śivēna', 'hindi': 'शिवेन', 'sandhi': 'Shivena', 'subwords': 'śiva'} {'word': 'śivēna', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'With Shiva', 'meaning_idx': 1}
	# subword: {'word': 'śiva', 'hindi': ' शिव ', 'sandhi': 'Shiva'} {'word': 'śiva', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Lord Shiva'}
{'word': 'śiva', 'hindi': ' शिव ', 'sandhi': 'Shiva'} {'word': 'śiva', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'Lord Shiva'}
#
# mainword: {'word': 'saha', 'hindi': 'सह', 'sandhi': 'Saha', 'subwords': 'saha'} {'word': 'saha', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php', 'meaning_value': 'In Company', 'meaning_idx': 5}
	# subword: {'word': 'saha', 'hindi': ' सह ', 'sandhi': 'Saha'} {'word': 'saha', 'source': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram

In [196]:
meta_info = SongIndex_sql.read_entry(**song_info)[0]#['song_idx']
print(meta_info)
# song_idx = 
total_no_verse = meta_info['total_slokas']
# for sloka_no in range(1,total_no_verse+1)
sloka_no = 1 # 



{'song_idx': 90, 'song_name': 'Shiva Panchakshara Stotram', 'song_short_name': 'ShivaPanchaksharaStotram', 'sloka_statues': 1, 'division': None, 'division_no': 0, 'total_slokas': 6, 'groups': None, 'devotion_god': 'Devi Kamakshi', 'author': 'Sri Adi Shankaracharya', 'describtion': None, 'other_links': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php'}


In [197]:
def get_synonyms_byverse(mySongs_sql,slokas_no,song_idx): # meta_info['song_idx']
    verse_line_list = mySongs_sql.read_entry(*['sloka_eng'],song_idx=song_idx,slokas_no=sloka_no)[0]['sloka_eng'].split('\n')
    # print(verse_line_list)
    if len(verse_line_list)==4:
        verse_line_list[1] =verse_line_list[1].split('|')[0].strip()
        verse_line_list[3] =verse_line_list[3].split('|')[0].strip()
    elif len(verse_line_list)==2:
        verse_line_list[0] =verse_line_list[0].split('|')[0].strip()
        verse_line_list[1] =verse_line_list[1].split('|')[0].strip()        
    # print(verse_line_list)
    output = ''
    for line_idx in verse_line_list:
        # for words in line_idx.split(" "):
        for word in line_idx.split(" "): #words:
            # word = words[4]
#             if '_'  in word:
#                 word = word.replace("_",'')
            print(word)
            print('DictonaryWord_sql.read_entry(word=word): ',DictonaryWord_sql.read_entry(word=word))
            
            if len(DictonaryWord_sql.read_entry(word=word)):
                data = DictonaryWord_sql.read_entry(word=word)[0]
                # print(data)
                if isinstance(data['subwords'],str):
                    for subword in data['subwords'].split('-'):
                        output += f"""{subword}={DictonaryMeaning_sql.read_entry(word=subword)[0]['meaning_value']}; """
                        print(f"""{subword}={DictonaryMeaning_sql.read_entry(word=subword)[0]['meaning_value']};""", end=" ")
                output += f"""{data['word']}={DictonaryMeaning_sql.read_entry(word=data['word'])[0]['meaning_value']}; """ 
                print(f"""{data['word']}={DictonaryMeaning_sql.read_entry(word=data['word'])[0]['meaning_value']};""", end =" ")
        output= output.strip() + '\n'
    return output.strip()#)
# get_synonyms_byverse(mySongs_sql,sloka_no,meta_info['song_idx'])   
get_synonyms_byverse(mySongs_sql,3,meta_info['song_idx'])   

nāgēndrahārāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'nāgēndrahārāya', 'phonetics': None, 'word_idx': 6011, 'hindi': 'नागेन्द्रहाराय', 'english': None, 'sandhi': 'Naagendra-Haaraaya', 'subwords': 'nāga-indra-hāra', 'abbrevations': None}]
nāga=Snake, Serpent; indra=Best, Excellent; hāra=Garland; nāgēndrahārāya=; trilocanāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'trilocanāya', 'phonetics': None, 'word_idx': 6012, 'hindi': 'त्रिलोचनाय', 'english': None, 'sandhi': 'Tri-Locanaaya', 'subwords': 'tri-locana', 'abbrevations': None}]
tri=Three; locana=Eye; trilocanāya=; bhasmāṅgarāgāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'bhasmāṅgarāgāya', 'phonetics': None, 'word_idx': 6013, 'hindi': 'भस्माङ्गरागाय', 'english': None, 'sandhi': 'Bhasma-Angga-Raagaaya', 'subwords': 'bhasman-aṅga-rāga', 'abbrevations': None}]
bhasman=Ashes; aṅga=Arms,Limbs; rāga=Colour, Act of Colouring; bhasmāṅgarāgāya=; mahēśvarāya
DictonaryWord_sql.read_entry(word=word):  [{'word': '

'nāga=Snake, Serpent; indra=Best, Excellent; hāra=Garland; nāgēndrahārāya=; tri=Three; locana=Eye; trilocanāya=;\nbhasman=Ashes; aṅga=Arms,Limbs; rāga=Colour, Act of Colouring; bhasmāṅgarāgāya=; mahā=Great; īśvara=God; mahēśvarāya=;\nnitya=Always; nityāya=; śuddha=Pure; śuddhāya=; diś=Direction; ambara=Clothes, Garments; digambarāya=;\ntasmai=; tasmai=; na_kārāya=; namas=Bow, Obeisance, Reverential Salutation; namaḥ=Salutations; śiva=Auspicious; śivāya=;'

In [198]:
meta_info = SongIndex_sql.read_entry(**song_info)[0]#['song_idx']
print(meta_info)
total_no_verse = meta_info['total_slokas']
for sloka_no in range(1,total_no_verse+1):
# sloka_no = 1 # 
    print(sloka_no)
    if mySongs_sql.read_entry(song_idx=meta_info['song_idx'],slokas_no=sloka_no)[0]['synonyms'] is  None:
        synonyms_by_verse = get_synonyms_byverse(mySongs_sql,sloka_no,meta_info['song_idx'])
        mySongs_sql.update_entry( *['song_idx','slokas_no'],synonyms=synonyms_by_verse, song_idx=meta_info['song_idx'], slokas_no=sloka_no)
    else:
        print("In MySong Table Synonyms entry is not Null")

{'song_idx': 90, 'song_name': 'Shiva Panchakshara Stotram', 'song_short_name': 'ShivaPanchaksharaStotram', 'sloka_statues': 1, 'division': None, 'division_no': 0, 'total_slokas': 6, 'groups': None, 'devotion_god': 'Devi Kamakshi', 'author': 'Sri Adi Shankaracharya', 'describtion': None, 'other_links': 'https://greenmesg.org/stotras/shiva/shiva_panchakshara_stotram.php'}
1
nāgēndrahārāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'nāgēndrahārāya', 'phonetics': None, 'word_idx': 6011, 'hindi': 'नागेन्द्रहाराय', 'english': None, 'sandhi': 'Naagendra-Haaraaya', 'subwords': 'nāga-indra-hāra', 'abbrevations': None}]
nāga=Snake, Serpent; indra=Best, Excellent; hāra=Garland; nāgēndrahārāya=; trilocanāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'trilocanāya', 'phonetics': None, 'word_idx': 6012, 'hindi': 'त्रिलोचनाय', 'english': None, 'sandhi': 'Tri-Locanaaya', 'subwords': 'tri-locana', 'abbrevations': None}]
tri=Three; locana=Eye; trilocanāya=; bhasmāṅgarāgāya
DictonaryWord

śiva=Auspicious; śivāya=; 4
vaśiṣṭhakumbhodbhavagautamārya_
DictonaryWord_sql.read_entry(word=word):  []
mūnīndradēvārcitaśēkharāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'mūnīndradēvārcitaśēkharāya', 'phonetics': None, 'word_idx': 6046, 'hindi': 'मूनीन्द्रदेवार्चितशेखराय', 'english': None, 'sandhi': 'Muuni-Indra-Deva-Aarcita-Shekharaaya', 'subwords': 'muni-indra-dēva-arcita-śēkhara', 'abbrevations': None}]
muni=Muni, Sage; indra=Best, Excellent; dēva=Deva, God; arcita=Worshipped; śēkhara=Head; mūnīndradēvārcitaśēkharāya=; candrārkavaiśvānaralocanāya
DictonaryWord_sql.read_entry(word=word):  [{'word': 'candrārkavaiśvānaralocanāya', 'phonetics': None, 'word_idx': 6047, 'hindi': 'चन्द्रार्कवैश्वानरलोचनाय', 'english': None, 'sandhi': 'Candra-Aarka-Vaishvaanara-Locanaaya', 'subwords': 'candra-arka-vaiśvānara-locana', 'abbrevations': None}]
candra=Moon; arka=Sun; vaiśvānara=Fire; locana=Eye; candrārkavaiśvānaralocanāya=; tasmai
DictonaryWord_sql.read_entry(word=word):  [{'word'

In [13]:
meta_info = SongIndex_sql.read_entry(**song_info)[0]#['song_idx']
total_no_verse = meta_info['total_slokas']
# for sloka_no in range(1,total_no_verse+1)
sloka_no = 1 # 


True